# Purpose
The purpose of this notebook is to explore the usage of a Conversational Retrieval Agent from `langchain` for our use case.  The agent will allow the LLM to determine when to query the database and when not to.  I'm mainly working from the following examples: [example from documentation](https://python.langchain.com/docs/use_cases/question_answering/how_to/conversational_retrieval_agents), [blog post example](https://blog.langchain.dev/conversational-retrieval-agents/).

# Vector Store

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local("data/vector-db/", embeddings)

# Retriever Tool
We need to define a retriever tool for the LLM to use to retrieve context.  The name and description of the tool are important because they are used by the LLM to determine when to use the tool.

In [2]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    db.as_retriever(), 
    "search_finance_opinions",
    "Searches and returns documents regarding Reddit user's opinions on topics related to finance and investing."
)
tools = [tool]

# Simple Agent Example

Here I'll use the built-in `create_conversational_retrieval_agent` from langchain, which just needs the tool provided above along with an LLM to use.  The agent uses `OpenAIFunctionsAgent` so I'll use an OpenAI chat model as the LLM.

In [3]:
import toml
import os

os.environ["OPENAI_API_KEY"] = toml.load("secrets.toml")["OPENAI_API_TOKEN"]

In [4]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

## Test Chat

Now I can test it out and monitor the decision making.

In [5]:
result = agent_executor({"input":"Hi my name is Coy and I'd like some advice."})
result



> Entering new AgentExecutor chain...
Hi Coy! I'm here to help. What kind of advice are you looking for?

> Finished chain.


{'input': "Hi my name is Coy and I'd like some advice.",
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False)],
 'output': "Hi Coy! I'm here to help. What kind of advice are you looking for?",
 'intermediate_steps': []}

In [6]:
result = agent_executor({"input":"I'd like to know what people are saying about Robinhood"})
result



> Entering new AgentExecutor chain...

Invoking: `search_finance_opinions` with `{'arg1': 'Robinhood'}`


[Document(page_content='CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they caused in losses. It might take a couple of days, but send Robinhood to the ground and GME to the moon./n/nChapman Albin is an investors rights firm that my buddy works at. Just got off the phone w him. He is going to post a press release regarding the case they are filing. \nLet me know if you need help finding a lawyer. \nDisclaimer: I’m not getting anything out of this', metadata={'id': 'l6x130'}), Document(page_content="Wallstreet Bets Set to Private Megathread/n/nThe moderators there

{'input': "I'd like to know what people are saying about Robinhood",
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False),
  HumanMessage(content="I'd like to know what people are saying about Robinhood", additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "arg1": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off u

In [7]:
result = agent_executor({"input":"So is it still safe to use?"})
result



> Entering new AgentExecutor chain...
The safety of using Robinhood or any other financial platform depends on various factors, including your personal risk tolerance and the specific features and security measures offered by the platform. It's important to conduct your own research and consider the following:

1. Security Measures: Look into the security measures implemented by Robinhood, such as two-factor authentication, encryption, and account protection features. Ensure that the platform has robust security protocols in place to protect your personal and financial information.

2. Regulatory Compliance: Check if Robinhood is regulated by relevant financial authorities in your country. Regulatory oversight can provide an additional layer of protection for users.

3. User Reviews and Reputation: Read reviews and feedback from other users to get an idea of their experiences with Robinhood. Consider both positive and negative reviews to make an informed decision.

4. Financial Stabi

{'input': 'So is it still safe to use?',
 'chat_history': [HumanMessage(content="Hi my name is Coy and I'd like some advice.", additional_kwargs={}, example=False),
  AIMessage(content="Hi Coy! I'm here to help. What kind of advice are you looking for?", additional_kwargs={}, example=False),
  HumanMessage(content="I'd like to know what people are saying about Robinhood", additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "arg1": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they ca

# Agent from Components

Now I'll build the agent by each component to exert a little more control.

## Memory
Any of the `langchain` memory objects can be used to store the chat history.  Here I'll use `AgentTokenBufferMemory` because I want the agent to remember the conversation in addition to the previous intermediate steps taken.

In [8]:
from langchain.agents.openai_functions_agent.agent_token_buffer_memory import AgentTokenBufferMemory

# This is needed for both the memory and the prompt
memory_key = "history"
memory = AgentTokenBufferMemory(memory_key=memory_key, llm=llm)

## Prompt Template
Now I'll set up the prompt template for the agent.  Since I'm using OpenAI for the agent and LLM, I'll use the default OpenAI Agent prompt provided by `OpenAIFunctionsAgent.create_prompt`.  I'll supply a custom system message, along with the chat history and previous intermediate steps.

In [9]:
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.messages import SystemMessage
from langchain.prompts import MessagesPlaceholder

system_message = SystemMessage(
        content=(
            "You are an expert on the opinions of Reddit Users."
            "Do your best to answer the questions. "
            "Feel free to use any tools available to look up "
            "relevant information, only if neccessary"
        )
)

prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
    )

## Tools

In addition to the retrieval tool defined above, I'm going to add a search tool here to showcase the capabilities of the agent.  This allows the agent to search the internet for additional context which it cannot retrieve from the vector store we supplied.  There are many options for internet search tools and I've chosen `DuckDuckGoSearchRun` because it doesn't require a key or any additional setup.

The agent uses the descriptions (and titles?) of the tools to decide which tool to use and when, so it is important these are descriptive and succinct.  

In [10]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.tools import Tool

search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool.from_function(
    func=search.run,
    name="DuckDuckGo_Search",
    description="Useful for when you need to do a search on the internet to find information that another tool can't find. be specific with your input."
)

tools.append(duckduckgo_tool)

## Build Agent

Now that I've defined each component above, building the agent is as simple as combining each component and instantiating, using `create_conversational_retrieval_agent`.

In [11]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

## Test Chat

Now I'll have a quick test chat with the agent and see when and if it chooses to use each tool.

In [12]:
result = agent_executor({"input": "Hi, I'm Coy"})
result



> Entering new AgentExecutor chain...
Hello Coy! How can I assist you today?

> Finished chain.


{'input': "Hi, I'm Coy",
 'chat_history': [HumanMessage(content="Hi, I'm Coy", additional_kwargs={}, example=False),
  AIMessage(content='Hello Coy! How can I assist you today?', additional_kwargs={}, example=False)],
 'output': 'Hello Coy! How can I assist you today?',
 'intermediate_steps': []}

In [13]:
result = agent_executor({"input": "What are people saying about Robinhood?"})
result



> Entering new AgentExecutor chain...

Invoking: `search_finance_opinions` with `{'query': 'Robinhood'}`


[Document(page_content='CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they caused in losses. It might take a couple of days, but send Robinhood to the ground and GME to the moon./n/nChapman Albin is an investors rights firm that my buddy works at. Just got off the phone w him. He is going to post a press release regarding the case they are filing. \nLet me know if you need help finding a lawyer. \nDisclaimer: I’m not getting anything out of this', metadata={'id': 'l6x130'}), Document(page_content="Wallstreet Bets Set to Private Megathread/n/nThe moderators ther

{'input': 'What are people saying about Robinhood?',
 'chat_history': [HumanMessage(content="Hi, I'm Coy", additional_kwargs={}, example=False),
  AIMessage(content='Hello Coy! How can I assist you today?', additional_kwargs={}, example=False),
  HumanMessage(content='What are people saying about Robinhood?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "query": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they caused in losses. It might take a couple of days, but send Robinh

In [14]:
result = agent_executor({"input":"So is it still safe to use?"})
result



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGo_Search` with `{'query': 'Is it safe to use Robinhood?'}`


Is Robinhood safe? Commission-free trades and a nifty mobile app made Robinhood a splash with investors. However, some wonder if it is safe to use, legitimate, or FINRA insured. Is... Advertiser disclosure Robinhood Review 2023: Pros, Cons and How It Compares Robinhood will appeal to investors looking to cut costs. But customer support is lacking, and the broker has run... In addition to my personal experience showing it is safe, you can rest assured that your first $500,000 of cash and stocks is insured by the SIPC. According to their website "Robinhood Financial LLC and Robinhood Securities, LLC are members of SIPC so securities in Robinhood accounts are protected up to $500,000. Robinhood Broker Review 4.4 U.S. News Overall Rating By Barbara Friedberg June 27, 2023, at 9:17 a.m. Our Verdict Robinhood has become one of the most popular brokerages in the U.S., than

{'input': 'So is it still safe to use?',
 'chat_history': [HumanMessage(content="Hi, I'm Coy", additional_kwargs={}, example=False),
  AIMessage(content='Hello Coy! How can I assist you today?', additional_kwargs={}, example=False),
  HumanMessage(content='What are people saying about Robinhood?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_finance_opinions', 'arguments': '{\n  "query": "Robinhood"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content=\'CLASS ACTION AGAINST ROBINHOOD. Allowing people to only sell is the definition of market manipulation. A class action must be started, Robinhood has made plenty of money off selling info about our trades to the hedge funds to be able to pay out a little for causing people to loose money now/n/nLEAVE ROBINHOOD. They dont deserve to make money off us after the millions they caused in losses. It might take a couple of days, but send Robinhood to the g